- Fine-tuning ParsBERT on PersianQA and ParSQUAD Datasets 

[Model and Datasets Path](https://drive.google.com/drive/folders/1_lgylP6jWCZGiqmvJWO-u7NOoqpD7LUp?usp=sharing)

# Setup

In [ ]:
!pip install -q transformers
!pip install -q datasets

In [ ]:
import json
import pandas as pd

In [ ]:
from datasets import load_dataset, load_from_disk, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_checkpoint = "HooshvareLab/bert-fa-base-uncased"
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.
batch_size = 8
lr = 3e-5
epoch = 2
DRIVE_PATH = "/content/drive/MyDrive/NLP_Final/"


# Load and Process Data

In [ ]:
train_path = DRIVE_PATH + 'Datasets/train_merged.json'
dev_path = DRIVE_PATH + 'Datasets/dev_merged.json'
test_path = DRIVE_PATH + 'Datasets/test_merged.json'


In [ ]:
with open(train_path,'r',encoding='utf-8') as f:
  train_data = json.load(f)

with open(dev_path,'r',encoding='utf-8') as f:
  dev_data = json.load(f)

with open(test_path,'r',encoding='utf-8') as f:
  test_data = json.load(f)


In [ ]:
train_df = pd.DataFrame(columns=['context','question','answers'])
for row in train_data:
  for par in row['paragraphs']:
    context = par['context']
    qas = par['qas']
    for qa in qas:
      question = qa['question']
      starts = []
      texts = []
      for ans in qa['answers']:
        if len(ans) != 0:
          answer_start = ans['answer_start']
          answer_text = ans['text']
        else:
          answer_start = ''
          answer_text = ''
        starts.append(answer_start)
        texts.append(answer_text)
      train_df = train_df.append({'context': context, 'question': question, 'answers': {'answer_start': starts, 'text': texts}}, ignore_index=True)


In [ ]:
dev_df = pd.DataFrame(columns=['context','question','answers'])
for row in dev_data:
  for par in row['paragraphs']:
    context = par['context']
    qas = par['qas']
    for qa in qas:
      question = qa['question']
      starts = []
      texts = []
      for ans in qa['answers']:
        if len(ans) != 0:
          answer_start = ans['answer_start']
          answer_text = ans['text']
        else:
          answer_start = ''
          answer_text = ''
        starts.append(answer_start)
        texts.append(answer_text)
      dev_df = dev_df.append({'context': context, 'question': question, 'answers': {'answer_start': starts, 'text': texts}}, ignore_index=True)


In [ ]:
test_df = pd.DataFrame(columns=['context','question','answers'])
for row in test_data:
  for par in row['paragraphs']:
    context = par['context']
    qas = par['qas']
    for qa in qas:
      question = qa['question']
      starts = []
      texts = []
      for ans in qa['answers']:
        if len(ans) != 0:
          answer_start = ans['answer_start']
          answer_text = ans['text']
        else:
          answer_start = ''
          answer_text = ''
        starts.append(answer_start)
        texts.append(answer_text)
      test_df = test_df.append({'context': context, 'question': question, 'answers': {'answer_start': starts, 'text': texts}}, ignore_index=True)


In [ ]:
print(train_df.shape)

(68010, 3)


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_df = train_df.to_csv(DRIVE_PATH + 'Datasets/train.csv')
dev_df = dev_df.to_csv(DRIVE_PATH + 'Datasets/dev.csv')
test_df = test_df.to_csv(DRIVE_PATH + 'Datasets/test.csv')

In [ ]:
train_dataset.save_to_disk("train.hf")
dev_dataset.save_to_disk("dev.hf")
test_dataset.save_to_disk("test.hf")

In [ ]:
!cp -r '/content/train.hf' "/content/drive/MyDrive/NLP_Final/Datasets/train.hf"
!cp -r '/content/test.hf' "/content/drive/MyDrive/NLP_Final/Datasets/test.hf"
!cp -r '/content/dev.hf' "/content/drive/MyDrive/NLP_Final/Datasets/dev.hf"

# Load Datasets

In [ ]:
train_dataset = load_from_disk(DRIVE_PATH + "Datasets/train.hf").shuffle(seed=42)
dev_dataset = load_from_disk(DRIVE_PATH + "Datasets/dev.hf").shuffle(seed=42)
test_dataset = load_from_disk(DRIVE_PATH + "Datasets/test.hf").shuffle(seed=42)

In [ ]:
print(len(train_dataset))

68010


# Tokenize Datasets

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
# the datasets library does cashing itself, batched is multitreading for fast-tokenizer
tokenized_train = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_dev = dev_dataset.map(prepare_train_features, batched=True, remove_columns=dev_dataset.column_names)
tokenized_test = test_dataset.map(prepare_train_features, batched=True, remove_columns=test_dataset.column_names)

  0%|          | 0/6 [00:00<?, ?ba/s]

# Training 

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
args = TrainingArguments(
    DRIVE_PATH + f"checkpoints2",
    save_strategy = 'epoch',
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.0001) 

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer)

In [ ]:
# start training
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 68086
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 17022


Epoch,Training Loss,Validation Loss
1,1.382700,1.529800


***** Running Evaluation *****
  Num examples = 5959
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-8511
Configuration saved in /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-8511/config.json
Model weights saved in /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-8511/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-8511/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-8511/special_tokens_map.json


Epoch,Training Loss,Validation Loss
1,1.382700,1.529800
2,0.822400,1.609336


***** Running Evaluation *****
  Num examples = 5959
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-17022
Configuration saved in /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-17022/config.json
Model weights saved in /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-17022/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-17022/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Final/checkpoints2/checkpoint-17022/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=17022, training_loss=1.2338444949905203, metrics={'train_runtime': 8082.9396, 'train_samples_per_second': 16.847, 'train_steps_per_second': 2.106, 'total_flos': 1.88476616843136e+16, 'train_loss': 1.2338444949905203, 'epoch': 2.0})

# Test Model

In [ ]:
!pip install -q sentencepiece

from tqdm import tqdm
from IPython.display import clear_output
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer


     |████████████████████████████████| 1.3 MB 3.2 MB/s 


In [ ]:
DRIVE_PATH = "/content/drive/MyDrive/NLP_Final/"
model_path = DRIVE_PATH + f"checkpoints2/checkpoint-17022/"  ## load model trained for 2 epochs
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [ ]:
class AnswerPredictor:
  def __init__(self, model, tokenizer, device='cuda', n_best=10, max_length=512, stride=256, no_answer=False):
      """Initializes PyTorch Question Answering Prediction
      It's best to leave use the default values.
      Args:
          model: Fine-tuned torch model
          tokenizer: Transformers tokenizer
          device (torch.device): Running device
          n_best (int): Number of best possible answers
          max_length (int): Tokenizer max length
          stride (int): Tokenizer stride
          no_answer (bool): If True, model can return "no answer"
      """
      self.model = model.eval().to(device)
      self.tokenizer = tokenizer
      self.device = device
      self.max_length = max_length
      self.stride = stride
      self.no_answer = no_answer
      self.n_best = n_best


  def model_pred(self, questions, contexts, batch_size=1):
      n = len(contexts)
      if n%batch_size!=0:
          raise Exception("batch_size must be divisible by sample length")

      tokens = self.tokenizer(questions, contexts, add_special_tokens=True, 
                              return_token_type_ids=True, return_tensors="pt", padding=True, 
                              return_offsets_mapping=True, truncation="only_second", 
                              max_length=self.max_length, stride=self.stride)

      start_logits, end_logits = [], []
      for i in tqdm(range(0, n-batch_size+1, batch_size)):
          with torch.no_grad():
              out = self.model(tokens['input_ids'][i:i+batch_size].to(self.device), 
                          tokens['attention_mask'][i:i+batch_size].to(self.device), 
                          tokens['token_type_ids'][i:i+batch_size].to(self.device))

              start_logits.append(out.start_logits)
              end_logits.append(out.end_logits)

      return tokens, torch.stack(start_logits).view(n, -1), torch.stack(end_logits).view(n, -1)


  def __call__(self, questions, contexts, batch_size=1, answer_max_len=100):
      """Creates model prediction
      
      Args: 
          questions (list): Question strings
          contexts (list): Contexts strings
          batch_size (int): Batch size
          answer_max_len (int): Sets the longests possible length for any answer
        
      Returns:
          dict: The best prediction of the model
              (e.g {0: {"text": str, "score": int}})
      """
      questions = [questions]
      contexts = [contexts]
      tokens, starts, ends = self.model_pred(questions, contexts, batch_size=batch_size)
      start_indexes = starts.argsort(dim=-1, descending=True)[:, :self.n_best]
      end_indexes = ends.argsort(dim=-1, descending=True)[:, :self.n_best]

      preds = {}
      for i, (c, q) in enumerate(zip(contexts, questions)):  
          min_null_score = starts[i][0] + ends[i][0] # 0 is CLS Token
          start_context = tokens['input_ids'][i].tolist().index(self.tokenizer.sep_token_id)
          
          offset = tokens['offset_mapping'][i]
          valid_answers = []
          for start_index in start_indexes[i]:
              # Don't consider answers that are in questions
              if start_index<start_context:
                  continue
              for end_index in end_indexes[i]:
                  # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                  # to part of the input_ids that are not in the context.
                  if (start_index >= len(offset) or end_index >= len(offset)
                      or offset[start_index] is None or offset[end_index] is None):
                      continue
                  # Don't consider answers with a length that is either < 0 or > max_answer_length.
                  if end_index < start_index or (end_index-start_index+1) > answer_max_len:
                      continue

                  start_char = offset[start_index][0]
                  end_char = offset[end_index][1]
                  valid_answers.append({"score": (starts[i][start_index] + ends[i][end_index]).item(),
                                        "text": c[start_char: end_char]})
                  
          if len(valid_answers) > 0:
              best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
          else:
              best_answer = {"text": "", "score": min_null_score}

          if self.no_answer:
              preds[i] = best_answer if best_answer["score"] >= min_null_score else {"text": "", "score": min_null_score}
          else:
              preds[i] = best_answer

      return preds


In [ ]:
predictor = AnswerPredictor(model, tokenizer, device='cuda', n_best=10, no_answer=True)


In [ ]:
test_dataset[9]

{'context': 'BSkyB سرویس HDTV خود را با نام Sky + HD در 22 مه 2006 راه اندازی کرد. قبل از راه اندازی ، BSkyB ادعا کرد که 40،000 نفر برای دریافت سرویس HD ثبت نام کرده اند. در هفته قبل از راه اندازی ، شایعاتی مبنی بر اینکه BSkyB در تهیه جعبه بالا (STB) خود از تولید کننده تامسون مشکل دارد ، آشکار شد. در روز پنجشنبه 18 مه 2006 ، و در آخر هفته قبل از راه اندازی ، مردم گزارش دادند که BSkyB نصب آن را لغو کرده یا دوباره برنامه ریزی کرده است. سرانجام ، بی بی سی گزارش داد که 17000 مشتری هنوز به دلیل تحویل ناموفق ، خدمات دریافت نکرده اند. در تاریخ 31 مارس 2012 ، آسمان تعداد کل خانه های دارای Sky + HD را 4222000 اعلام کرد.',
 'question': 'چه زمانی BSkyB سرویس HDTV خود را راه اندازی کرد؟',
 'answers': {'answer_start': [43, 40, 43],
  'text': ['22 مه 2006', 'در 22 مه 2006', '22 مه 2006']}}

In [ ]:
context = test_dataset[9]['context'] 
question = test_dataset[9]['question'] 
answer = test_dataset[9]['answers']['text']
print(answer)


preds = predictor(question, context, batch_size=1)
pred = preds[0]['text'].strip()
print('\n\n Model Prediction: ', preds[0]['text'].strip())
print(preds)

# print(compute_exact_match(answer, pred))
# print(compute_f1(answer, pred))

['22 مه 2006', 'در 22 مه 2006', '22 مه 2006']


100%|██████████| 1/1 [00:00<00:00, 75.37it/s]



 Model Prediction:  22 مه 2006
{0: {'score': 17.336265563964844, 'text': '22 مه 2006'}}
